## Week4
### 【問題1】コンペティション内容の確認

コンペティションのOverviewページ読み、「Home Credit Default Risk」について以下の観点について確認してください。


何を学習し、何を予測するのか


 →　各顧客の特徴量（年齢、年収等）から返済できるかどうかを学習し返済有無を予測する。

どのようなファイルを作りKaggleに提出するか

　→テストセットの各SK_ID_CURRについて、TARGET変数の確率を予測する

提出されたものはどういった指標値で評価されるのか

　→予測された確率と観測された目標の間のROC曲線の下の領域で評価
   ( from sklearn.metrics import roc_auc_score )


### 【問題2】学習と検証
データを簡単に分析、前処理し、学習、検証するまでの一連の流れを作成・実行してください。


検証にはこのコンペティションで使用される評価指標を用いるようにしてください。学習に用いる手法は指定しません。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# データ取得
train_data = pd.read_csv("application_train.csv")
pd.set_option('display.max_columns', 122)
pd.set_option('display.max_rows', 122)
train_data

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.0220,0.0198,0.0,0.0000,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.0000,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.0790,0.0554,0.0,0.0000,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.0100,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [2]:
# TARGETとの相関係数を確認
corr_matrix = train_data.corr()
corr_matrix["TARGET"].nlargest(10)

TARGET                         1.000000
DAYS_BIRTH                     0.078239
REGION_RATING_CLIENT_W_CITY    0.060893
REGION_RATING_CLIENT           0.058899
DAYS_LAST_PHONE_CHANGE         0.055218
DAYS_ID_PUBLISH                0.051457
REG_CITY_NOT_WORK_CITY         0.050994
FLAG_EMP_PHONE                 0.045982
REG_CITY_NOT_LIVE_CITY         0.044395
FLAG_DOCUMENT_3                0.044346
Name: TARGET, dtype: float64

ベースラインとしては、特徴量にターゲットとの相関係数が大きい"DAYS_BIRTH","REGION_RATING_CLIENT_W_CITY","REGION_RATING_CLIENT"の3つとし、k近傍法で学習・評価をすることとした。

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score 

# 特徴量とラベルを抽出
X = train_data[["DAYS_BIRTH","REGION_RATING_CLIENT_W_CITY","REGION_RATING_CLIENT"]]
y = train_data["TARGET"]

# データをホールドアウト法で分割
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.75,stratify=y,random_state=1)

# 標準化
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scalered = scaler.transform(X_train)
X_test_scalered = scaler.transform(X_test)

# k近傍法でモデルの学習
model = KNeighborsClassifier(n_neighbors=100)
model.fit(X_train_scalered,y_train)
predicted_label = model.predict_proba(X_test_scalered)
predicted_label = predicted_label[:,1]

# 評価
roc_auc_score(y_test,predicted_label)

0.5744054250139229

### 【問題3】テストデータに対する推定
テストデータ（application_test.csv）に対して推定を行い、Kaggleに提出を行ってください。


正しく提出が行えていれば、精度は低くても構いません。

In [4]:
test_data = pd.read_csv("application_test.csv")
test_data 

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,Unaccompanied,Working,Higher education,Married,House / apartment,0.018850,-19241,-2329,-5170.0,-812,NaN,1,1,0,1,0,1,NaN,2.0,2,2,TUESDAY,18,0,0,0,0,0,0,Kindergarten,0.752614,0.789654,0.159520,0.0660,0.0590,0.9732,NaN,NaN,NaN,0.1379,0.1250,NaN,NaN,NaN,0.0505,NaN,NaN,0.0672,0.0612,0.9732,NaN,NaN,NaN,0.1379,0.1250,NaN,NaN,NaN,0.0526,NaN,NaN,0.0666,0.0590,0.9732,NaN,NaN,NaN,0.1379,0.1250,NaN,NaN,NaN,0.0514,NaN,NaN,NaN,block of flats,0.0392,"Stone, brick",No,0.0,0.0,0.0,0.0,-1740.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,Unaccompanied,Working,Secondary / secondary special,Married,House / apartment,0.035792,-18064,-4469,-9118.0,-1623,NaN,1,1,0,1,0,0,Low-skill Laborers,2.0,2,2,FRIDAY,9,0,0,0,0,0,0,Self-employed,0.564990,0.291656,0.432962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,NaN,Working,Higher education,Married,House / apartment,0.019101,-20038,-4458,-2175.0,-3503,5.0,1,1,0,1,0,0,Drivers,2.0,2,2,MONDAY,14,0,0,0,0,0,0,Transport: type 3,NaN,0.699787,0.610991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-856.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,Unaccompanied,Working,Secondary / secondary special,Marr

In [5]:
# 特徴量を抽出
X = test_data[["DAYS_BIRTH","REGION_RATING_CLIENT_W_CITY","REGION_RATING_CLIENT"]]

# 標準化
X_scalered = scaler.transform(X)

# 学習済モデルで予測
predicted_label = model.predict_proba(X_scalered)
predicted_label = predicted_label[:,1].reshape(-1,1)

# 提出できる形にIDとラベルの予測値（確率）を結合する
ID = np.array(test_data["SK_ID_CURR"]).reshape(-1,1)
submission1_values = np.concatenate([ID,predicted_label],axis=1)
submission1_values = pd.DataFrame(submission1_values,columns=["SK_ID_CURR","TARGET"])
submission1_values["SK_ID_CURR"] = submission1_values["SK_ID_CURR"].astype("int32")
print(submission1_values.head())

# csvに変換
submission1_values.to_csv('submission1.csv',index=False)

   SK_ID_CURR  TARGET
0      100001    0.09
1      100005    0.05
2      100013    0.04
3      100028    0.07
4      100038    0.13


スコア：0.57607

### 【問題4】特徴量エンジニアリング
精度を上げるために以下のような観点で 特徴量エンジニアリング（Feature Engineering） を行ってください。


どの特徴量を使うか
どう前処理をするか

何をした時に検証データに対する評価指標がどのようになったかをまとめてください。最低5パターンの学習・検証を行ってください。


精度が高かったものに関してはテストデータに対しても推定を行い、Kaggleに提出を行ってください。

初めに特徴量を全て（数値データのみ）を使用し確認する。

In [7]:
only_num = train_data.select_dtypes(include='number')

In [10]:
only_num = only_num.dropna(how='any')

In [11]:
# 特徴量とラベルを抽出
X = only_num.drop(["TARGET"],axis=1)
y = only_num["TARGET"]

# データをホールドアウト法で分割
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.75,stratify=y,random_state=1)

# 標準化
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scalered = scaler.transform(X_train)
X_test_scalered = scaler.transform(X_test)

# k近傍法でモデルの学習
model = KNeighborsClassifier(n_neighbors=100)
model.fit(X_train_scalered,y_train)
predicted_label = model.predict_proba(X_test_scalered)
predicted_label = predicted_label[:,1]

# 評価
roc_auc_score(y_test,predicted_label)

0.6459258382175911

スコアは0.64という結果となった。ここから特徴量を選択してどのように変化するか確認する。
前回で相関係数が正に大きい特徴量は確認したので以下を確認する。

>相関係数が-1に近い特徴量を利用する

In [13]:
corr_matrix["TARGET"].nsmallest(10)

EXT_SOURCE_3                 -0.178919
EXT_SOURCE_2                 -0.160472
EXT_SOURCE_1                 -0.155317
DAYS_EMPLOYED                -0.044932
FLOORSMAX_AVG                -0.044003
FLOORSMAX_MEDI               -0.043768
FLOORSMAX_MODE               -0.043226
AMT_GOODS_PRICE              -0.039645
REGION_POPULATION_RELATIVE   -0.037227
ELEVATORS_AVG                -0.034199
Name: TARGET, dtype: float64

In [14]:
X2.isnull().sum()　　#欠損値の確認

SyntaxError: invalid character in identifier (<ipython-input-14-08e39e2bcbad>, line 1)

In [20]:
X2 = train_data.dropna(how="any")
X2.isnull().sum()

SK_ID_CURR                      0
TARGET                          0
NAME_CONTRACT_TYPE              0
CODE_GENDER                     0
FLAG_OWN_CAR                    0
FLAG_OWN_REALTY                 0
CNT_CHILDREN                    0
AMT_INCOME_TOTAL                0
AMT_CREDIT                      0
AMT_ANNUITY                     0
AMT_GOODS_PRICE                 0
NAME_TYPE_SUITE                 0
NAME_INCOME_TYPE                0
NAME_EDUCATION_TYPE             0
NAME_FAMILY_STATUS              0
NAME_HOUSING_TYPE               0
REGION_POPULATION_RELATIVE      0
DAYS_BIRTH                      0
DAYS_EMPLOYED                   0
DAYS_REGISTRATION               0
DAYS_ID_PUBLISH                 0
OWN_CAR_AGE                     0
FLAG_MOBIL                      0
FLAG_EMP_PHONE                  0
FLAG_WORK_PHONE                 0
FLAG_CONT_MOBILE                0
FLAG_PHONE                      0
FLAG_EMAIL                      0
OCCUPATION_TYPE                 0
CNT_FAM_MEMBER

ここではEXT_SOURCE_3、EXT_SOURCE_2 、EXT_SOURCE_1　を用いて学習を行う。

In [22]:
# 特徴量とラベルを抽出
train_data = train_data.dropna(how='any') # 欠損値があるため削除
X2 = train_data[["EXT_SOURCE_3","EXT_SOURCE_2","EXT_SOURCE_1"]]
y2 = train_data["TARGET"]

# データをホールドアウト法で分割
X2_train,X2_test,y2_train,y2_test = train_test_split(X2,y2,train_size=0.75,stratify=y2,random_state=1)

# 標準化
scaler = StandardScaler()
scaler.fit(X2_train)
X2_train_scalered = scaler.transform(X2_train)
X2_test_scalered = scaler.transform(X2_test)

# k近傍法でモデルの学習
model_2 = KNeighborsClassifier(n_neighbors=100)
model_2.fit(X2_train_scalered,y2_train)
predicted_label_2 = model_2.predict_proba(X2_test_scalered)
predicted_label_2 = predicted_label_2[:,1]

# 評価
roc_auc_score(y2_test,predicted_label_2)

0.7261714470109715

スコアは前回の0.574よりも上昇した。これは、相関係数の絶対値が大きい特徴量を使用したためめと考えられる。"EXT_SOURCE_3","EXT_SOURCE_2","EXT_SOURCE_1"

次に、相関係数が正で最も大きい"DAYS_BIRTH"、負で最も小さい（絶対値大きい）"EXT_SOURCE_3","EXT_SOURCE_2","EXT_SOURCE_1"を組み合わせて確認する。

In [23]:
# 特徴量とラベルを抽出
train_data = train_data.dropna(how='any') # 欠損値があるため削除
X3 = train_data[["DAYS_BIRTH","EXT_SOURCE_3","EXT_SOURCE_2","EXT_SOURCE_1"]]
y3 = train_data["TARGET"]

# データをホールドアウト法で分割
X3_train,X3_test,y3_train,y3_test = train_test_split(X3,y3,train_size=0.75,stratify=y3,random_state=1)

# 標準化
scaler = StandardScaler()
scaler.fit(X3_train)
X3_train_scalered = scaler.transform(X3_train)
X3_test_scalered = scaler.transform(X3_test)

# k近傍法でモデルの学習
model_3 = KNeighborsClassifier(n_neighbors=100)
model_3.fit(X3_train_scalered,y3_train)
predicted_label_3 = model_3.predict_proba(X3_test_scalered)
predicted_label_3 = predicted_label_3[:,1]

# 評価
roc_auc_score(y3_test,predicted_label_3)

0.7241752592792713

"DAYS_BIRTH""EXT_SOURCE_3","EXT_SOURCE_2","EXT_SOURCE_1"の組み合わせた特徴量でもある程度高いスコア0.724を確認できた。
既存の特徴量でターゲットとの相関係数が高い特徴量で学習させてるとある程度高いスコアを確認することができた。

最後にこの"EXT_SOURCE_3","EXT_SOURCE_2","EXT_SOURCE_1"については各特徴量の掛け算や各の平均したものを特徴量として生成した。
また、年齢単位（'DAYS_BIRTH'）や　金属年数単位'DAYS_EMPLOYED'、年収単位で　代表的な特徴量を生成した。こちらを加えて確認をしてみる。

In [27]:
df= pd.read_csv("application_train.csv")

df['EXT_SOURCE_3'] = df['EXT_SOURCE_3'].fillna(df['EXT_SOURCE_3'].mean())
df['EXT_SOURCE_2'] = df['EXT_SOURCE_2'].fillna(df['EXT_SOURCE_2'].mean())
df['EXT_SOURCE_1'] = df['EXT_SOURCE_1'].fillna(df['EXT_SOURCE_1'].mean())
df['NEW_EXT_SOURCES_MEAN'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
df['NEW_SCORES_STD'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
df['NEW_SCORES_STD'] = df['NEW_SCORES_STD'].fillna(df['NEW_SCORES_STD'].mean())


df['NEW_CAR_TO_BIRTH_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
df['NEW_CAR_TO_BIRTH_RATIO'] = df['NEW_CAR_TO_BIRTH_RATIO'].fillna(df['NEW_CAR_TO_BIRTH_RATIO'].mean())
df['NEW_PHONE_TO_BIRTH_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']
df['NEW_PHONE_TO_BIRTH_RATIO'] = df['NEW_PHONE_TO_BIRTH_RATIO'].fillna(df['NEW_PHONE_TO_BIRTH_RATIO'].mean())

df['NEW_EMPLOY_TO_BIRTH_RATIO'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']

df['NEW_CREDIT_TO_INCOME_RATIO'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']

In [28]:
train = df[["DAYS_BIRTH","EXT_SOURCE_3","EXT_SOURCE_2","EXT_SOURCE_1",'NEW_EXT_SOURCES_MEAN','NEW_SCORES_STD','NEW_PHONE_TO_BIRTH_RATIO','NEW_EMPLOY_TO_BIRTH_RATIO','NEW_PHONE_TO_BIRTH_RATIO','NEW_CREDIT_TO_INCOME_RATIO']]

In [29]:
df.isnull().sum()

SK_ID_CURR                    0
TARGET                        0
NAME_CONTRACT_TYPE            0
CODE_GENDER                   0
FLAG_OWN_CAR                  0
                             ..
NEW_SCORES_STD                0
NEW_CAR_TO_BIRTH_RATIO        0
NEW_PHONE_TO_BIRTH_RATIO      0
NEW_EMPLOY_TO_BIRTH_RATIO     0
NEW_CREDIT_TO_INCOME_RATIO    0
Length: 128, dtype: int64

In [30]:
# 特徴量とラベルを抽出
X4 = train
y4 = df["TARGET"]

# データをホールドアウト法で分割
X4_train,X4_test,y4_train,y4_test = train_test_split(X4,y4,train_size=0.75,stratify=y4,random_state=1)

# 標準化
scaler = StandardScaler()
scaler.fit(X4_train)
X4_train_scalered = scaler.transform(X4_train)
X4_test_scalered = scaler.transform(X4_test)

# k近傍法でモデルの学習
model_4 = KNeighborsClassifier(n_neighbors=100)
model_4.fit(X4_train_scalered,y4_train)
predicted_label_4 = model_4.predict_proba(X4_test_scalered)
predicted_label_4 = predicted_label_4[:,1]

# 評価
roc_auc_score(y4_test,predicted_label_4)

0.7153584565656919

スコア結果より、こちらでも同程度のスコアを確認することができた。今回はこの最後の学習モデルでkaggleに提出する。

In [31]:
df = pd.read_csv("application_test.csv")

df['EXT_SOURCE_3'] = df['EXT_SOURCE_3'].fillna(df['EXT_SOURCE_3'].mean())
df['EXT_SOURCE_2'] = df['EXT_SOURCE_2'].fillna(df['EXT_SOURCE_2'].mean())
df['EXT_SOURCE_1'] = df['EXT_SOURCE_1'].fillna(df['EXT_SOURCE_1'].mean())
df['NEW_EXT_SOURCES_MEAN'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
df['NEW_SCORES_STD'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
df['NEW_SCORES_STD'] = df['NEW_SCORES_STD'].fillna(df['NEW_SCORES_STD'].mean())


df['NEW_CAR_TO_BIRTH_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
df['NEW_PHONE_TO_BIRTH_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']
df['NEW_PHONE_TO_BIRTH_RATIO'] = df['NEW_PHONE_TO_BIRTH_RATIO'].fillna(df['NEW_PHONE_TO_BIRTH_RATIO'].mean())

df['NEW_EMPLOY_TO_BIRTH_RATIO'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']

df['NEW_CREDIT_TO_INCOME_RATIO'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']

In [32]:
# 特徴量を抽出
X = df[["DAYS_BIRTH","EXT_SOURCE_3","EXT_SOURCE_2","EXT_SOURCE_1",'NEW_EXT_SOURCES_MEAN','NEW_SCORES_STD','NEW_PHONE_TO_BIRTH_RATIO','NEW_EMPLOY_TO_BIRTH_RATIO','NEW_PHONE_TO_BIRTH_RATIO','NEW_CREDIT_TO_INCOME_RATIO']]

# 標準化
X_scalered = scaler.transform(X)

# 学習済モデルで予測
predicted_label = model_4.predict_proba(X_scalered)
predicted_label = predicted_label[:,1].reshape(-1,1)

# 提出できる形にIDとラベルの予測値（確率）を結合する
ID = np.array(df["SK_ID_CURR"]).reshape(-1,1)
submission2_values = np.concatenate([ID,predicted_label],axis=1)
submission2_values = pd.DataFrame(submission2_values,columns=["SK_ID_CURR","TARGET"])
submission2_values["SK_ID_CURR"] = submission2_values["SK_ID_CURR"].astype("int32")
print(submission2_values.head())

# csvに変換
submission2_values.to_csv('submission2.csv',index=False)

   SK_ID_CURR  TARGET
0      100001    0.02
1      100005    0.19
2      100013    0.02
3      100028    0.06
4      100038    0.18


スコア：0.69537

### ※自主取り組み（参考）

In [34]:
df= pd.read_csv("application_train.csv")

df['EXT_SOURCE_3'] = df['EXT_SOURCE_3'].fillna(df['EXT_SOURCE_3'].mean())
df['EXT_SOURCE_2'] = df['EXT_SOURCE_2'].fillna(df['EXT_SOURCE_2'].mean())
df['EXT_SOURCE_1'] = df['EXT_SOURCE_1'].fillna(df['EXT_SOURCE_1'].mean())
df['NEW_EXT_SOURCES_MEAN'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
df['NEW_SCORES_STD'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
df['NEW_SCORES_STD'] = df['NEW_SCORES_STD'].fillna(df['NEW_SCORES_STD'].mean())


df['NEW_CAR_TO_BIRTH_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
df['NEW_CAR_TO_BIRTH_RATIO'] = df['NEW_CAR_TO_BIRTH_RATIO'].fillna(df['NEW_CAR_TO_BIRTH_RATIO'].mean())
df['NEW_PHONE_TO_BIRTH_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']
df['NEW_PHONE_TO_BIRTH_RATIO'] = df['NEW_PHONE_TO_BIRTH_RATIO'].fillna(df['NEW_PHONE_TO_BIRTH_RATIO'].mean())

df['NEW_EMPLOY_TO_BIRTH_RATIO'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']

df['NEW_CREDIT_TO_INCOME_RATIO'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']

In [35]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(n_splits=4)
score_probs_gbm = []

X = df[["DAYS_BIRTH","EXT_SOURCE_3","EXT_SOURCE_2","EXT_SOURCE_1",'NEW_EXT_SOURCES_MEAN','NEW_SCORES_STD','NEW_PHONE_TO_BIRTH_RATIO','NEW_EMPLOY_TO_BIRTH_RATIO','NEW_PHONE_TO_BIRTH_RATIO','NEW_CREDIT_TO_INCOME_RATIO']]
y = df["TARGET"]

for tr_idx, va_idx in kf.split(X,y):
    # 学習データとバリデーションに分ける
    tr_X, va_X = X.iloc[tr_idx], X.iloc[va_idx]
    tr_y, va_y = y.iloc[tr_idx], y.iloc[va_idx]
    
    # 標準化
    sscaler = StandardScaler()
    sscaler.fit(tr_X)
    tr_X_scalered = sscaler.transform(tr_X)
    va_X_scalered = sscaler.transform(va_X)
    
    # lightgbm が扱うデータセットの形式に直す
    dtrain = lgb.Dataset(tr_X_scalered, label=tr_y)
    dtest = lgb.Dataset(va_X_scalered, label=va_y, reference= dtrain)
    
    #モデル学習
    params= {'objective':'binary','learning_rate':0.1,'reg_alpha':0,'leg_lambda':1,'min_child_weight':1,'max_depth':5,'subsample':0.8,'colsample':0.8,'colsample_bytree':0.8,'ramdom_state':71}
    gbm = lgb.train(params,
                    dtrain,
                    valid_sets=dtest,
                    num_boost_round=100,
                    verbose_eval=10
                    )
    
    #確率予測を返す
    predicted_label_gbm = gbm.predict(va_X_scalered)
    
    #スコア算出
    score_gbm = roc_auc_score(va_y,predicted_label_gbm)
    
    #スコアを記録
    score_probs_gbm.append(score_gbm)

print(score_probs_gbm)
print(np.mean(score_probs_gbm))

/Users/mishibatoshihiro/.pyenv/versions/anaconda3-2019.10/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


[10]	valid_0's binary_logloss: 0.25839
[20]	valid_0's binary_logloss: 0.254631
[30]	valid_0's binary_logloss: 0.253487
[40]	valid_0's binary_logloss: 0.253007
[50]	valid_0's binary_logloss: 0.252704
[60]	valid_0's binary_logloss: 0.252629
[70]	valid_0's binary_logloss: 0.252511
[80]	valid_0's binary_logloss: 0.252481
[90]	valid_0's binary_logloss: 0.252521
[100]	valid_0's binary_logloss: 0.252549
[10]	valid_0's binary_logloss: 0.258437
[20]	valid_0's binary_logloss: 0.254678
[30]	valid_0's binary_logloss: 0.253477
[40]	valid_0's binary_logloss: 0.252967
[50]	valid_0's binary_logloss: 0.252678
[60]	valid_0's binary_logloss: 0.252469
[70]	valid_0's binary_logloss: 0.252343
[80]	valid_0's binary_logloss: 0.252305
[90]	valid_0's binary_logloss: 0.252262
[100]	valid_0's binary_logloss: 0.252196
[10]	valid_0's binary_logloss: 0.258656
[20]	valid_0's binary_logloss: 0.254824
[30]	valid_0's binary_logloss: 0.253725
[40]	valid_0's binary_logloss: 0.253136
[50]	valid_0's binary_logloss: 0.252888

In [37]:
df = pd.read_csv("application_test.csv")

df['EXT_SOURCE_3'] = df['EXT_SOURCE_3'].fillna(df['EXT_SOURCE_3'].mean())
df['EXT_SOURCE_2'] = df['EXT_SOURCE_2'].fillna(df['EXT_SOURCE_2'].mean())
df['EXT_SOURCE_1'] = df['EXT_SOURCE_1'].fillna(df['EXT_SOURCE_1'].mean())
df['NEW_EXT_SOURCES_MEAN'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis=1)
df['NEW_SCORES_STD'] = df[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].std(axis=1)
df['NEW_SCORES_STD'] = df['NEW_SCORES_STD'].fillna(df['NEW_SCORES_STD'].mean())


df['NEW_CAR_TO_BIRTH_RATIO'] = df['OWN_CAR_AGE'] / df['DAYS_BIRTH']
df['NEW_CAR_TO_BIRTH_RATIO'] = df['NEW_CAR_TO_BIRTH_RATIO'].fillna(df['NEW_CAR_TO_BIRTH_RATIO'].mean())
df['NEW_PHONE_TO_BIRTH_RATIO'] = df['DAYS_LAST_PHONE_CHANGE'] / df['DAYS_BIRTH']
df['NEW_PHONE_TO_BIRTH_RATIO'] = df['NEW_PHONE_TO_BIRTH_RATIO'].fillna(df['NEW_PHONE_TO_BIRTH_RATIO'].mean())

df['NEW_EMPLOY_TO_BIRTH_RATIO'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']

df['NEW_CREDIT_TO_INCOME_RATIO'] = df['AMT_CREDIT'] / df['AMT_INCOME_TOTAL']

In [38]:
# 特徴量を抽出
X = df[["DAYS_BIRTH","EXT_SOURCE_3","EXT_SOURCE_2","EXT_SOURCE_1",'NEW_EXT_SOURCES_MEAN','NEW_SCORES_STD','NEW_PHONE_TO_BIRTH_RATIO','NEW_EMPLOY_TO_BIRTH_RATIO','NEW_PHONE_TO_BIRTH_RATIO','NEW_CREDIT_TO_INCOME_RATIO']]

# 標準化
X_scalered = scaler.transform(X)

# 学習済モデルで予測
predicted_label = gbm.predict(X_scalered)
predicted_label = predicted_label.reshape(-1,1)

# 提出できる形にIDとラベルの予測値（確率）を結合する
ID = np.array(df["SK_ID_CURR"]).reshape(-1,1)
submission3_values = np.concatenate([ID,predicted_label],axis=1)
submission3_values = pd.DataFrame(submission3_values,columns=["SK_ID_CURR","TARGET"])
submission3_values["SK_ID_CURR"] = submission3_values["SK_ID_CURR"].astype("int32")
print(submission3_values.head())

# csvに変換
submission3_values.to_csv('submission3.csv',index=False)

   SK_ID_CURR    TARGET
0      100001  0.036551
1      100005  0.069452
2      100013  0.033297
3      100028  0.047649
4      100038  0.128504


提出スコア：0.71046